In [1]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
# For tf.dataset
AUTO = tf.data.experimental.AUTOTUNE

In [2]:
train = pd.read_csv('../input/stanford-dog/train.csv')
test = pd.read_csv('../input/stanford-dog/test.csv')

print(train.shape, test.shape)
train.head()

(12000, 3) (8580, 3)


,filename,anno_filename,Category
0,n02085620-Chihuahua/n02085620_5927.jpg,n02085620-Chihuahua/n02085620_5927,1
1,n02085620-Chihuahua/n02085620_4441.jpg,n02085620-Chihuahua/n02085620_4441,1
2,n02085620-Chihuahua/n02085620_1502.jpg,n02085620-Chihuahua/n02085620_1502,1
3,n02085620-Chihuahua/n02085620_1916.jpg,n02085620-Chihuahua/n02085620_1916,1
4,n02085620-Chihuahua/n02085620_13151.jpg,n02085620-Chihuahua/n02085620_13151,1


In [3]:
train['Labels'] = train.Category - 1
test['Labels'] = test.Category - 1

train.head()

,filename,anno_filename,Category,Labels
0,n02085620-Chihuahua/n02085620_5927.jpg,n02085620-Chihuahua/n02085620_5927,1,0
1,n02085620-Chihuahua/n02085620_4441.jpg,n02085620-Chihuahua/n02085620_4441,1,0
2,n02085620-Chihuahua/n02085620_1502.jpg,n02085620-Chihuahua/n02085620_1502,1,0
3,n02085620-Chihuahua/n02085620_1916.jpg,n02085620-Chihuahua/n02085620_1916,1,0
4,n02085620-Chihuahua/n02085620_13151.jpg,n02085620-Chihuahua/n02085620_13151,1,0


In [4]:
tpath = "../input/stanford-dog/images/Images"
train_paths = train.filename.apply(lambda x: os.path.join(tpath, x))
train_paths.values

array(['../input/stanford-dog/images/Images/n02085620-Chihuahua/n02085620_5927.jpg',
       '../input/stanford-dog/images/Images/n02085620-Chihuahua/n02085620_4441.jpg',
       '../input/stanford-dog/images/Images/n02085620-Chihuahua/n02085620_1502.jpg',
       ...,
       '../input/stanford-dog/images/Images/n02116738-African_hunting_dog/n02116738_6754.jpg',
       '../input/stanford-dog/images/Images/n02116738-African_hunting_dog/n02116738_9333.jpg',
       '../input/stanford-dog/images/Images/n02116738-African_hunting_dog/n02116738_2503.jpg'],
      dtype=object)

In [5]:
from tensorflow.keras.utils import to_categorical

# convert to one-hot-encoding-labels
train_labels = to_categorical(train.Labels)
train_labels.shape

(12000, 120)

In [6]:
tpath = "../input/stanford-dog/images/Images"
test_paths = test.filename.apply(lambda x: os.path.join(tpath, x))
test_paths.values

array(['../input/stanford-dog/images/Images/n02085620-Chihuahua/n02085620_2650.jpg',
       '../input/stanford-dog/images/Images/n02085620-Chihuahua/n02085620_4919.jpg',
       '../input/stanford-dog/images/Images/n02085620-Chihuahua/n02085620_1765.jpg',
       ...,
       '../input/stanford-dog/images/Images/n02116738-African_hunting_dog/n02116738_3635.jpg',
       '../input/stanford-dog/images/Images/n02116738-African_hunting_dog/n02116738_2988.jpg',
       '../input/stanford-dog/images/Images/n02116738-African_hunting_dog/n02116738_6330.jpg'],
      dtype=object)

In [7]:
def decode_image(filename, label=None, image_size=(299, 299)):
    bits = tf.io.read_file(filename)
    image = tf.image.decode_jpeg(bits, channels=3)
    image = tf.cast(image, tf.float32) / 255.0
    image = tf.image.resize(image, image_size)
    
    if label is None:
        return image
    else:
        return image, label

In [8]:
def augment(image, label=None):
    image = tf.image.random_flip_left_right(image)
    
    if label is None:
        return image
    else:
        return image, label

In [9]:
batch_size = 32

train_dataset = (
    tf.data.Dataset
    .from_tensor_slices((train_paths, train_labels))
    .map(decode_image, num_parallel_calls=AUTO)
    .map(augment, num_parallel_calls=AUTO)
    .repeat()
    .shuffle(2048)
    .batch(batch_size)
    .prefetch(AUTO)
)

test_dataset = (tf.data.Dataset
        .from_tensor_slices(test_paths)
        .map(decode_image, num_parallel_calls=AUTO)
        .batch(batch_size)
)

In [10]:
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam

# from efficientnet.tfkeras import EfficientNetB2

model = tf.keras.Sequential([
        ResNet101(weights = 'imagenet', 
                       include_top = False,
                       input_shape = (299, 299, 3)),
        GlobalAveragePooling2D(),
        Dense(1024, activation = 'relu'),
        Dropout(0.25),
        Dense(512, activation = 'relu'),
        Dropout(0.25),
        Dense(120, activation = 'softmax')
])
    
model.compile(optimizer = 'sgd',
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'])
    
model.summary()

171450368/171446536 [==============================] - 3s 0us/step
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet101 (Model)            (None, 10, 10, 2048)      42658176  
_________________________________________________________________
global_average_pooling2d (Gl (None, 2048)              0         
_________________________________________________________________
dense (Dense)                (None, 1024)              2098176   
_________________________________________________________________
dropout (Dropout)            (None, 1024)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               524800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_______________________________________________________

In [11]:
def Train_model(model, batch_size, EPOCHS):
    n_steps = train_labels.shape[0] // batch_size
    EPOCHS = EPOCHS
    
    Model = model
    history = Model.fit(train_dataset, 
                    steps_per_epoch = n_steps,
                    epochs = EPOCHS,
                    verbose = 1)
    return Model

In [12]:
batch_size = 32
EPOCHS = 50

def check(x, y):
    if x == y:
        return 1
    else:
        return 0
    
print('Training')
model = Train_model(model, batch_size, EPOCHS)
pred = model.predict(test_dataset, verbose=1)
    
test['Prediction'] = pred.argmax(axis=1)
test['Score'] = test.apply(lambda x: check(x['Prediction'], x['Labels']), axis=1)
print('Test accuracy on iterations is ', 100 * test.Score.sum() / test.shape[0])

Training
Epoch 1/50
375/375 [==============================] - 189s 505ms/step - loss: 2.8936 - accuracy: 0.3228
Epoch 2/50
375/375 [==============================] - 189s 504ms/step - loss: 1.3139 - accuracy: 0.6246
Epoch 3/50
375/375 [==============================] - 189s 505ms/step - loss: 0.8788 - accuracy: 0.7353
Epoch 4/50
375/375 [==============================] - 189s 504ms/step - loss: 0.6224 - accuracy: 0.8096
Epoch 5/50
375/375 [==============================] - 189s 505ms/step - loss: 0.4641 - accuracy: 0.8541
Epoch 6/50
375/375 [==============================] - 189s 505ms/step - loss: 0.3551 - accuracy: 0.8910
Epoch 7/50
375/375 [==============================] - 189s 505ms/step - loss: 0.2853 - accuracy: 0.9112
Epoch 8/50
375/375 [==============================] - 189s 505ms/step - loss: 0.2206 - accuracy: 0.9338
Epoch 9/50
375/375 [==============================] - 189s 504ms/step - loss: 0.1855 - accuracy: 0.9446
Epoch 10/50
375/375 [==============================] - 

In [13]:
from sklearn.metrics import classification_report, confusion_matrix

print(classification_report(test.Labels, test.Prediction))

              precision    recall  f1-score   support

           0       0.73      0.73      0.73        52
           1       0.85      0.88      0.87        85
           2       0.89      0.80      0.84       152
           3       0.73      0.78      0.75        49
           4       0.73      0.71      0.72       114
           5       0.95      0.86      0.90        88
           6       0.96      0.95      0.95        96
           7       0.79      0.72      0.75        72
           8       0.65      0.57      0.61        72
           9       0.96      0.95      0.96       139
          10       0.82      0.85      0.84        75
          11       0.90      0.75      0.82        95
          12       0.93      0.80      0.86        87
          13       0.83      0.85      0.84        71
          14       0.76      0.92      0.83        59
          15       0.50      0.60      0.55        53
          16       0.69      0.70      0.70        57
          17       0.61    